In [55]:
#env verification

import sys
sys.executable

'd:\\Sentio\\venv\\Scripts\\python.exe'

In [56]:
#loading dataset

import pandas as pd
df = pd.read_csv("../data/emotional_dataset.csv")
df.head()

,Unnamed: 0,Situation,emotion,empathetic_dialogues,labels,Unnamed: 5,Unnamed: 6
0,0,I remember going to the fireworks with my best...,sentimental,Customer :I remember going to see the firework...,"Was this a friend you were in love with, or ju...",NaN,NaN
1,1,I remember going to the fireworks with my best...,sentimental,Customer :This was a best friend. I miss her.\...,Where has she gone?,NaN,NaN
2,2,I remember going to the fireworks with my best...,sentimental,Customer :We no longer talk.\nAgent :,Oh was this something that happened because of...,NaN,NaN
3,3,I remember going to the fireworks with my best...,sentimental,Customer :Was this a friend you were in love w...,This was a best friend. I miss her.,NaN,NaN
4,4,I remember going to the fireworks with my best...,sentimental,Customer :Where has she gone?\nAgent :,We no longer talk.,NaN,NaN


In [57]:
#checking for null values

df.shape
df.columns

Index(['Unnamed: 0', 'Situation', 'emotion', 'empathetic_dialogues', 'labels',
       'Unnamed: 5', 'Unnamed: 6'],
      dtype='str')

In [58]:
#droping useless columns

df = df.drop(columns=["Unnamed: 5", "Unnamed: 6"])
df.head()

,Unnamed: 0,Situation,emotion,empathetic_dialogues,labels
0,0,I remember going to the fireworks with my best...,sentimental,Customer :I remember going to see the firework...,"Was this a friend you were in love with, or ju..."
1,1,I remember going to the fireworks with my best...,sentimental,Customer :This was a best friend. I miss her.\...,Where has she gone?
2,2,I remember going to the fireworks with my best...,sentimental,Customer :We no longer talk.\nAgent :,Oh was this something that happened because of...
3,3,I remember going to the fireworks with my best...,sentimental,Customer :Was this a friend you were in love w...,This was a best friend. I miss her.
4,4,I remember going to the fireworks with my best...,sentimental,Customer :Where has she gone?\nAgent :,We no longer talk.


In [59]:
#renaming columns

df = df.rename(columns={
    "Situation": "situation",
    "emotion": "emotion",
    "empathetic_dialogues": "dialogue",
    "labels": "response"
})

df.columns

Index(['Unnamed: 0', 'situation', 'emotion', 'dialogue', 'response'], dtype='str')

In [60]:
#clean dialogue text

df["dialogue"] = (
    df["dialogue"]
    .str.replace("Customer:", "", regex=False)
    .str.replace("Agent:", "", regex=False)
    .str.replace("\n", " ", regex=False)
    .str.strip()
)

df["dialogue"].head()

0    Customer :I remember going to see the firework...
1    Customer :This was a best friend. I miss her. ...
2                 Customer :We no longer talk. Agent :
3    Customer :Was this a friend you were in love w...
4                Customer :Where has she gone? Agent :
Name: dialogue, dtype: str

In [61]:
#checking for null values

df.isnull().sum()

Unnamed: 0    0
situation     0
emotion       4
dialogue      0
response      0
dtype: int64

In [62]:
#removing null rows

df = df.dropna()
df.shape

(64632, 5)

In [63]:
#10-core emotional taxonomy

emotion_map = {
    # Joy
    "joyful": "joy", "happy": "joy", "excited": "joy",
    "content": "joy", "impressed": "joy",

    # Sadness
    "sad": "sadness", "lonely": "sadness",
    "nostalgic": "sadness", "disappointed": "sadness",

    # Anger
    "angry": "anger", "furious": "anger", "annoyed": "anger",

    # Fear
    "afraid": "fear", "anxious": "fear", "terrified": "fear",

    # Love / Attachment
    "sentimental": "love", "grateful": "love", "affectionate": "love",

    # Surprise
    "surprised": "surprise", "shocked": "surprise",

    # Disgust
    "disgusted": "disgust",

    # Guilt / Shame
    "guilty": "guilt", "ashamed": "guilt",

    # Pride / Confidence
    "proud": "pride", "confident": "pride",

    # Hope / Anticipation
    "hopeful": "hope", "anticipating": "hope", "prepared": "hope"
}

emotion_map["stressed"] = "fear"
emotion_map["stress"] = "fear"


df["emotion_normalized"] = df["emotion"].map(emotion_map).fillna("other")


In [64]:
#appling normalization

df["emotion_normalized"] = df["emotion"].map(emotion_map)
df = df.dropna(subset=["emotion_normalized"])

df["emotion_normalized"].value_counts()

emotion_normalized
joy         8325
sadness     8284
anger       6554
fear        6205
hope        5982
pride       4284
love        3864
guilt       3747
surprise    3295
disgust     2044
Name: count, dtype: int64

In [65]:
#creating RAG text

df["rag_text"] = (
    "Situation: " + df["situation"] +
    " | Emotion: " + df["emotion_normalized"] +
    " | Dialogue: " + df["dialogue"]
)

df[["rag_text", "response"]].head()


,rag_text,response
0,Situation: I remember going to the fireworks w...,"Was this a friend you were in love with, or ju..."
1,Situation: I remember going to the fireworks w...,Where has she gone?
2,Situation: I remember going to the fireworks w...,Oh was this something that happened because of...
3,Situation: I remember going to the fireworks w...,This was a best friend. I miss her.
4,Situation: I remember going to the fireworks w...,We no longer talk.


In [66]:
#loading sentence transformer model

from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-MiniLM-L6-v2")

Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [67]:
#creating embeddings for RAG text

texts = df["rag_text"].tolist()

embeddings = model.encode(
    texts,
    show_progress_bar=True,
    convert_to_numpy=True
)

embeddings.shape

Batches:   0%|          | 0/1644 [00:00<?, ?it/s]

(52584, 384)

In [68]:
#building the Vector Index

import faiss
import numpy as np

dimension = embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

index.ntotal

52584

In [78]:
#creating Retrieval Function (Core Engine)

# creating Retrieval Function (Core Engine)

def retrieve_similar(query, k=7, emotion=None):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k * 4)

    results = df.iloc[indices[0]].copy()
    results["similarity"] = distances[0]

    if emotion:
        emotion = emotion.lower()
        emotion = emotion_map.get(emotion, emotion)

        # soft emotion filtering (bias, not drop)
        emotion_matches = results[results["emotion_normalized"] == emotion]

        MIN_MATCHES = max(2, k // 2)

        if len(emotion_matches) >= MIN_MATCHES:
            results = emotion_matches
        # else: keep original results (fallback)

    return results[
        ["emotion_normalized", "situation", "dialogue", "response", "similarity"]
    ].head(k)


In [70]:
#emotion alias mapping

EMOTION_ALIASES = {
    "anxiety": "fear",
    "anxious": "fear",
    "stressed": "fear",
    "stress": "fear",
    "scared": "fear",

    "sad": "sadness",
    "sadness": "sadness",

    "angry": "anger",
    "anger": "anger",

    "happy": "joy",
    "happiness": "joy",
    "joyful": "joy",

    "love": "love",
    "hopeful": "hope",
    "proud": "pride",
    "disgusted": "disgust",
    "surprised": "surprise",
    "guilty": "guilt"
}


In [71]:
#Upgraded Retrieval Function with Emotion Filtering

def retrieve_similar(query, k=7, emotion=None):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, k * 3)

    results = df.iloc[indices[0]].copy()
    results["similarity"] = distances[0]

    if emotion:
        emotion = emotion.lower()
        emotion = EMOTION_ALIASES.get(emotion, emotion)
        results = results[results["emotion_normalized"] == emotion]


    return results[
        ["emotion_normalized", "situation", "dialogue", "response", "similarity"]
    ].head(k)


In [79]:
df["emotion_normalized"].value_counts(normalize=True)

emotion_normalized
joy         0.158318
sadness     0.157538
anger       0.124639
fear        0.118002
hope        0.113761
pride       0.081470
love        0.073482
guilt       0.071257
surprise    0.062662
disgust     0.038871
Name: proportion, dtype: float64

In [80]:
#test_0
retrieve_similar(
    "I have been stressed and overwhelmed lately",
    emotion="anxious"
)

,emotion_normalized,situation,dialogue,response,similarity
2264,fear,I am finding it hard to meet the project deadl...,Customer :I have a presentation to prepare for...,Keep calm.That is the very important step you ...,0.965705
54375,fear,i feel very stressed because of my job,Customer :i am i een started doing yoga to rel...,Try meditation. It worked well for me!,0.974169
2265,fear,I am finding it hard to meet the project deadl...,"Customer :I know, but somehow the crippling an...",You should breath deep and start focusing on t...,0.982964
1116,fear,Lately I have been very upset and worried abou...,Customer :My finances are not the greatest lat...,"I am sorry to hear that, I hope things turn ar...",1.007577
25689,fear,I have a big test coming up for school. I have...,"Customer :Oh no, that's always really stressfu...",Well I want to be as prepared as possible. I w...,1.009948
64547,fear,I find myself with being over ran with anxiety...,Customer :I totally understand. Life can some...,That makes sense. I am going to start trying t...,1.028930
50835,fear,I have a lot of nerves today! I am sick a lot ...,Customer :I am sick a lot in my health and I c...,Sounds like quite a problem! I think you shoul...,1.029644


In [81]:
#test_1

retrieve_similar(
    "I feel lonely these days",
    emotion="sad"
)

,emotion_normalized,situation,dialogue,response,similarity
2192,sadness,I felt lonely when I was in my senior year of ...,"Customer :Yes, that was some time ago. I'm mos...",I like keeping busy too. It really helps me st...,0.650294
13727,sadness,I live alone and I feel lonely a lot! I don't ...,Customer :I'm sorry that you feel lonely. Agent :,Thanks! I don't like it a lot but it's better ...,0.747666
34576,sadness,When my kids are in school all day I get lonel...,Customer :I can understand that. It's usually ...,Most of the time I'm happy and content but occ...,0.762892
9533,sadness,I feel lonely all the time. I live alone and m...,Customer :I often feel lonely. Agent :,Why do you think you often feel that way?,0.773335
30237,sadness,Sometimes I feel lonely. I am single and live ...,Customer :Sometimes I feel lonely. Agent :,why is that?,0.785132
4476,sadness,When I went to Texas for some office work. I w...,Customer :Have you ever felt lonely in your li...,I suppose I have. Why did you go to Texas?,0.785531
6389,sadness,Everyone has gone out for the evening and I am...,Customer :It is nights like this when I am alo...,Sorry to hear that. I'm in the same boat. At l...,0.807522


In [82]:
#test_2

retrieve_similar(
    "I feel proud of myself",
    emotion="confident"
)

,emotion_normalized,situation,dialogue,response,similarity
24159,pride,I feel proud of myself when I reach a new mile...,Customer :thats great you should feel proud Ag...,I set a target of getting my first victory roy...,0.767886
30373,pride,I was proud of a pizza I made yesterday,Customer :Hm delicious. Do you feel proud of y...,Absolutely! I'm a legend!,0.770670
24157,pride,I feel proud of myself when I reach a new mile...,Customer :I feel proud of myself when I reach ...,thats great you should feel proud,0.800708
15110,pride,I am so proud of myself. I started a new job ...,Customer :Oh YAY congrats! You should be proud...,I know. I had my last job 5 years so it was h...,0.802904
15108,pride,I am so proud of myself. I started a new job ...,Customer :Im so proud of myself. I started a ...,Oh YAY congrats! You should be proud of yourse...,0.905525
71,pride,I felt proud when I had accomplished a dream t...,"Customer :I'm not familiar with abseiling, but...","absailing, its when you rappeling down a build...",0.953992
1155,pride,I am so proud about myself. I just got promoted.,"Customer :""I am so proud about myself. I just ...",Good job! I am sure that took a lot of hard work!,0.967538
